<img src="../img/htw-logo.png" width=150>


**I758 Wissens- und KI-basierte Systeme**

# Lineare Regression mit Kreuzvalidierung
Quelle: Kaggle  / Anpassungen CK
 

<font color="green"><b>KLAUSURTAUGLICH.</b></font>
Dieses Notebook gehört zu den fünf Notebooks, die Sie für die Klausur einreichen können. Bei vollständiger und korrekter Bearbeitung **erhalten Sie Punkte für die Abgabe, die zu Ihrer Klausur addiert werden.**

<span style="color:#FF5F00"><b>AUFGABE:</b></span><br>

Auch dieses Notebook enthält wieder eine Reihe von neuen Bibliotheken. Legen Sie eine neue virtuelle Umgebung an oder erweitern Sie die bestehende `ml-class`. Studieren Sie dazu die Import-Anweisungen.


Tipp: Vermutlich wird Ihr Leben einfacher, wenn Sie eine `requirements.txt` anlegen. Mehr dazu finden Sie z.B. in [diesem Post von Stackoverflow](https://stackoverflow.com/questions/66899666/how-to-install-from-requirements-txt) 



### Cross-Validation with Linear Regression

This notebook demonstrates how to do feature selection and cross-validation (CV) with linear regression as an example (it is heavily used in almost all modelling techniques such as decision trees, SVM etc.). We will mainly use `sklearn` to do cross-validation.

In [ ]:
pip install scikit-learn

### Explore the data

Let us first load and prepare the data.

In [ ]:
# import all libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
import re

import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import scale
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline

import warnings # supress warnings
warnings.filterwarnings('ignore')

In [ ]:
# import Housing.csv
housing = pd.read_csv('data/Housing.csv', sep=";")
housing.head()

In [ ]:
# number of samples 
len(housing.index)

For the first experiment, we'll do regression with only one feature, `area`, and we'll build a model to predict the price. Let's filter the data so it only contains `area` and `price`.

<span style="color:#FF5F00"><b>AUFGABE:</b></span><br>
Reduzieren Sie den Data Frame.
</div>

Nutzen Sie die Pandas-Funktion `loc[ , ]`, um `housing` auf die Spalten `area` und `price` zu reduzieren. Speichern Sie das Ergebnis in `df`. Der Befehl `df.head()` sollte im Anschluss einen Output wie im Bild liefern.

<img src="./img/image_df_head.png">

In [ ]:
# filter only area and price
df = # your code here
df.head()

Scikit learn offers a wide variety of usefull *transformer* functions to scale, normalize and otherwise manipulate features.

<span style="color:#FF5F00"><b>AUFGABE:</b></span><br>

Skalieren Sie den Data Frame.
</div>

Nutzen Sie einen geeigneeten `Scaler`, um die beiden Spalten von `df` so zu skalieren, dass die Werte zwischen 0 und 1 liegen. Speichern Sie das Ergebnis in `df`. Der Befehl `df.head()` sollte im Anschluss einen Output wie im Bild liefern.

<img src="./img/image_df_scaler.png">

In [ ]:
# recaling the variables (both)
df_columns = # your code here

scaler = # your code here
df = # your code here

# rename columns (since now its an np array)
df = pd.DataFrame(df)
df.columns = df_columns
df.head()

Let's plot the data to see the relationship between our single input feauture and the target variable.

In [ ]:
# visualise area-price relationship
sns.regplot(x="area", y="price", data=df, fit_reg=False)

<span style="color:#FF5F00"><b>AUFGABE:</b></span><br>
Machen Sie sich klar, was Sie im Plot sehen. Was bedeuten die Werte an den Achsen? Was bedeutet die Verteilung der Punkte entlang x und y? Sieht man einen Zusammenhang zwischen x und y?
</div>

In [ ]:
# split into train and test
df_train, df_test = train_test_split(df, 
                                     train_size = 0.7, 
                                     test_size = 0.3, 
                                     random_state = 10)
print(len(df_train))
print(len(df_test))

### Experiments to Understand Overfitting

In this section, let's quickly go through some experiments to understand what overfitting looks like. We'll run some experiments using polynomial regression.

In [ ]:
# split into X and y for both train and test sets
# reshaping is required since sklearn requires the data to be in shape
# (n, 1), not as a series of shape (n, )
X_train = df_train['area']
X_train = X_train.values.reshape(-1, 1)
y_train = df_train['price']

X_test = df_test['area']
X_test = X_test.values.reshape(-1, 1)
y_test = df_test['price']

### Polynomial Regression

You already know simple linear regression:

$y = \beta_0 + \beta_1 x_1$

In polynomial regression of degree $n$, we fit a curve of the form:

$y = \beta_0 + \beta_1 x_1 + \beta_2x_1^2 + \beta_3x_1^3 ... + \beta_nx_1^n$

In the experiment below, we have fitted polynomials of various degrees on the housing data and compared their performance on train and test sets.

In sklearn, polynomial features can be generated using the `PolynomialFeatures` class. Also, to perform `LinearRegression` and `PolynomialFeatures` in tandem, we will use the module `sklearn_pipeline` - it basically creates the features and feeds the output to the model (in that sequence).

In [ ]:
len(X_train)

Let's now predict the y labels (for both train and test sets) and store the predictions in a table. Each row of the table is one data point, each column is a value of $n$ (degree).

<table style="width:100%">
  <tr>
    <th>   </th>
    <th>degree-1</th>
    <th>degree-2</th> 
    <th>degree-3</th>
    <th>...</th>
    <th>degree-n</th>
  </tr>
  <tr>
    <th>x1</th>
  </tr>
  <tr>
    <th>x2</th>
  </tr>
   <tr>
    <th>x3</th>
    </tr>
    <tr>
    <th>...</th>
    </tr>
    <tr>
    <th>xn</th>
    </tr>
</table>

In [ ]:
# fit multiple polynomial features
degrees = [1, 2, 3, 6, 10, 20]

# initialise y_train_pred and y_test_pred matrices to store the train and test predictions
# each row is a data point, each column a prediction using a polynomial of some degree
y_train_pred = np.zeros((len(X_train), len(degrees)))
y_test_pred = np.zeros((len(X_test), len(degrees)))

for i, degree in enumerate(degrees):
    
    # make pipeline: create features, then feed them to linear_reg model
    model = make_pipeline(PolynomialFeatures(degree), LinearRegression())
    model.fit(X_train, y_train)
    
    # predict on test and train data
    # store the predictions of each degree in the corresponding column
    y_train_pred[:, i] = model.predict(X_train)
    y_test_pred[:, i] = model.predict(X_test)
    

In [ ]:
# visualise train and test predictions
# note that the y axis is on a log scale

plt.figure(figsize=(16, 8))

# train data
plt.subplot(121)
plt.scatter(X_train, y_train)
plt.yscale('log')
plt.title("Train data")
for i, degree in enumerate(degrees):    
    plt.scatter(X_train, y_train_pred[:, i], s=15, label=str(degree))
    plt.legend(loc='upper left')
    
# test data
plt.subplot(122)
plt.scatter(X_test, y_test)
plt.yscale('log')
plt.title("Test data")
for i, degree in enumerate(degrees):    
    plt.scatter(X_test, y_test_pred[:, i], label=str(degree))
    plt.legend(loc='upper left')

<span style="color:#FF5F00"><b>AUFGABE:</b></span><br>
Machen Sie sich klar, was Sie in den Plots sehen. Notieren Sie im Feld unten Ihre Beobachtungen.
<ul>
<li>In welchem Zusammenhang stehen die bunten Linien zu den blauen Punkten?</li>
<li>Was bedeuten die verschiedenen Farben der Linien?</li>
<li>Können Sie aus den Linien (im Feld "Test") irgendetwas sehen zu der Frage, welches Modell das "Beste" sein könnte?</li>
</ul>
</div>

Ihre Antworten hier ...

Let us now evaluate the quality of our predictions by comparing predictions to true values. We use the R2 statistic for that.

In [ ]:
# compare r2 for train and test sets (for all polynomial fits)
print("R-squared values: \n")

for i, degree in enumerate(degrees):
    train_r2 = round(sklearn.metrics.r2_score(y_train, y_train_pred[:, i]), 2)
    test_r2 = round(sklearn.metrics.r2_score(y_test, y_test_pred[:, i]), 2)
    print("Polynomial degree {0}: train score={1}, test score={2}".format(degree, 
                                                                         train_r2, 
                                                                         test_r2))

<span style="color:#FF5F00"><b>AUFGABE:</b></span><br>

Schauen Sie sich die Zahlen mit den R squared values in Ruhe an. Bemerken Sie das absurde Verhalten?
</div>

### 1. Building a Model Without Cross-Validation

Let's now build a multiple regression model. First, let's build a vanilla MLR model without any cross-validation etc. 

In [ ]:
# data preparation

# list of all the "yes-no" binary categorical variables
# we'll map yes to 1 and no to 0
binary_vars_list =  ['mainroad', 'guestroom', 'basement', 'hotwaterheating', 'airconditioning', 'prefarea']

# defining the map function
def binary_map(x):
    return x.map({'yes': 1, "no": 0})

# applying the function to the housing variables list
housing[binary_vars_list] = housing[binary_vars_list].apply(binary_map)
housing.head()

In [ ]:
# 'dummy' variables
# get dummy variables for 'furnishingstatus' 
# also, drop the first column of the resulting df (since n-1 dummy vars suffice)
status = pd.get_dummies(housing['furnishingstatus'], drop_first = True)
status.head()

In [ ]:
# concat the dummy variable df with the main df
housing = pd.concat([housing, status], axis = 1)
housing.head()

In [ ]:
# 'furnishingstatus' since we alreday have the dummy vars
housing.drop(['furnishingstatus'], axis = 1, inplace = True)
housing.head()

#### Splitting Into Train and Test

<span style="color:#FF5F00"><b>AUFGABE:</b></span><br>

Das können Sie schon: erstellen Sie aus `housing` Test- und Trainingsdaten (70% Train, 30% Test).  Skalieren Sie anschließend alle numerischen Werte in Train mit einem `MinMaxScaler`!
</div>

In [ ]:
# train-test 70-30 split
df_train, df_test = # Ihr code hier

# rescale the features
scaler = MinMaxScaler()

# apply scaler() to all the numeric columns 
numeric_vars = ['area', 'bedrooms', 'bathrooms', 'stories', 'parking','price']
df_train[numeric_vars] = # Ihr code hier
df_train.head()

<span style="color:#FF5F00"><b>AUFGABE:</b></span><br>

Das können Sie auch schon: Übertragen Sie nun die Daten auf das Test-Set!
</div>

In [ ]:
# apply rescaling to the test set also
df_test[numeric_vars] = # Ihr Code hier
df_test.head()

In [ ]:
# divide into X_train, y_train, X_test, y_test
y_train = df_train.pop('price')
X_train = df_train

y_test = df_test.pop('price')
X_test = df_test

Note that we haven't rescaled the test set yet, which we'll need to do later while making predictions.

#### Using RFE 

Now, we have 13 predictor features. To build the model using Recursive Feature Elemination (RFE), we need to tell RFE how many features we want in the final model. It then runs a feature elimination algorithm. 

Note that the number of features to be used in the model is a **hyperparameter**.

In [ ]:
# num of max features
len(X_train.columns)

In [ ]:
# first model with an arbitrary choice of n_features
# running RFE with number of, let's say features=10

lm = LinearRegression()
lm.fit(X_train, y_train)

rfe = RFE(lm, n_features_to_select=10)             
rfe = rfe.fit(X_train, y_train)

In [ ]:
# tuples of (feature name, whether selected, ranking)
# note that the 'rank' is > 1 for non-selected features
list(zip(X_train.columns,rfe.support_,rfe.ranking_))

In [ ]:
# predict prices of X_test
y_pred = rfe.predict(X_test)

# evaluate the model on test set
r2 = sklearn.metrics.r2_score(y_test, y_pred)
print(r2)

<span style="color:#FF5F00"><b>AUFGABE:</b></span><br>

Halten Sie kurz inne und überlegen Sie, was Sie hier sehen. Halten Sie unten fest, welches Subset die RFE jetzt eigentlich ausgewählt hat.
</div>

Notieren Sie hier das Subset

In [ ]:
# try with another value of RFE
lm = LinearRegression()
lm.fit(X_train, y_train)

rfe = RFE(lm, n_features_to_select=6)             
rfe = rfe.fit(X_train, y_train)

# predict prices of X_test
y_pred = rfe.predict(X_test)
r2 = sklearn.metrics.r2_score(y_test, y_pred)
print(r2)

<div class="alert alert-block alert-success">
<b>Arbeitsauftrag ohne Dokumentation</b> 
Wir haben jetzt also eine RFE mit n=6 und eine RFE mit n=10 gemacht. Welche ist besser?
</div>

## 2. Problems in the Current Approach

In train-test split, we have three options:
1. **Simply split into train and test**: But that way tuning a hyperparameter makes the model 'see' the test data (i.e. knowledge of test data leaks into the model)
2. **Split into train, validation, test sets**: Then the validation data would eat into the training set
3. **Cross-validation**: Split into train and test, and train multiple models by sampling the train set. Finally, just test once on the test set.


## 3. Cross-Validation: A Quick Recap

The following figure illustrates k-fold cross-validation with k=4. There are some other schemes to divide the training set, we'll look at them briefly later.

![](https://chkra.github.io/wikisys_lecture/workshops/08/08/img/kfold_cross.png)

## 4. Cross-Validation in sklearn

Let's now experiment with k-fold CV.

### 4.1 K-Fold CV

In [ ]:
# k-fold CV (using all the 13 variables)
lm = LinearRegression()
scores = cross_val_score(lm, X_train, y_train, scoring='r2', cv=5)
scores      

In [ ]:
# the other way of doing the same thing (more explicit)

# create a KFold object with 5 splits 
folds = KFold(n_splits = 5, shuffle = True, random_state = 100)
scores = cross_val_score(lm, X_train, y_train, scoring='r2', cv=folds)
scores   

<span style="color:#FF5F00"><b>AUFGABE:</b></span><br>

Sie sollten 5 Werte in der Variable scores sehen. Überlegen Sie kurz: was bedeuten diese Werte?
</div>

In [ ]:
# can tune other metrics, such as MSE
scores = cross_val_score(lm, X_train, y_train, scoring='neg_mean_squared_error', cv=5)
scores

### 4.2 Hyperparameter Tuning Using Grid Search Cross-Validation

A common use of cross-validation is for tuning hyperparameters of a model. The most common technique is what is called **grid search** cross-validation.
![](https://chkra.github.io/wikisys_lecture/workshops/08/08/img/hyperparam.png)

In [ ]:
# number of features in X_train
len(X_train.columns)

In [ ]:
# step-1: create a cross-validation scheme
folds = KFold(n_splits = 5, shuffle = True, random_state = 100)

# step-2: specify range of hyperparameters to tune
hyper_params = [{'n_features_to_select': list(range(1, len(X_train.columns)))}]


# step-3: perform grid search
# 3.1 specify model
lm = LinearRegression()
lm.fit(X_train, y_train)
rfe = RFE(lm)             

# 3.2 call GridSearchCV()
model_cv = GridSearchCV(estimator = rfe, 
                        param_grid = hyper_params, 
                        scoring= 'r2', 
                        cv = folds, 
                        verbose = 1,
                        return_train_score=True)      

# fit the model
model_cv.fit(X_train, y_train)                  


In [ ]:
# cv results
cv_results = pd.DataFrame(model_cv.cv_results_)
cv_results

In [ ]:
# plotting cv results
plt.figure(figsize=(16,6))

plt.plot(cv_results["param_n_features_to_select"], cv_results["mean_test_score"])
plt.plot(cv_results["param_n_features_to_select"], cv_results["mean_train_score"])
plt.xlabel('number of features')
plt.ylabel('r-squared')
plt.title("Optimal Number of Features")
plt.legend(['test score', 'train score'], loc='upper left')

<span style="color:#FF5F00"><b>AUFGABE:</b></span><br>

Schauen Sie auf die Tabelle. Was ist das beste Ergebnis und in welcher Spalte / welcher Zeile sehen Sie das?
</div>

PS: Vielleicht hilft [dieser Link](https://stackoverflow.com/questions/54608088/what-is-gridsearch-cv-results-could-any-explain-all-the-things-in-that-i-e-me) ?

Now we can choose the optimal value of number of features and build a final model.

<span style="color:#FF5F00"><b>AUFGABE:</b></span><br>


Nutzen Sie den Wert aus der letzten Frage und korrigieren Sie bei Bedarf den Wert der Variable n_features_optimal.
</div>

In [ ]:
# final model
n_features_optimal = 18

lm = LinearRegression()
lm.fit(X_train, y_train)

rfe = RFE(lm, n_features_to_select=n_features_optimal)             
rfe = rfe.fit(X_train, y_train)

# predict prices of X_test
y_pred = lm.predict(X_test)
r2 = sklearn.metrics.r2_score(y_test, y_pred)
print(r2)

Notice that the test score is very close to the 'mean test score' on the k-folds (about 60%). In general, the mean score estimated by CV will usually be a good estimate of the test score. 

### 4.3 Types of Cross-Validation Schemes


1. **K-Fold** cross-validation: Most common
2. **Leave One Out (LOO)**: Takes each data point as the 'test sample' once, and trains the model on the rest n-1 data points. Thus, it trains n total models.
    - Advantage: Utilises the data well since each model is trained on n-1 samples
    - Disadvantage: Computationally expensive
3. **Leave P-Out (LPO)**: Creat all possible splits after leaving p samples out. For n data points, there are (nCp) possibile train-test splits.
4. (**For classification problems**) **Stratified K-Fold**: Ensures that the relative class proportion is approximately preserved in each train and validation fold. Important when ther eis huge class imbalance (e.g. 98% good customers, 2% bad).

#### Additional Reading ####
The sklearn documentation enlists all CV schemes <a href="http://scikit-learn.org/stable/modules/cross_validation.html">here.</a>
